# core

In [1]:
import src.StreamPort.core as core

anas = core.Analyses()
pm = core.ProcessingMethod()
mt = core.Metadata(description="test", version="1.0")
wf = core.Workflow()

print(anas)
print(pm)
print(mt)
print(wf)

core = core.Engine()
core.print()


Analyses 
  data_type:  
  formats: [] 
  data: 0 
  No data available. 


ProcessingMethod
  data_type    
  method       
  required     
  algorithm    
  input        
  output       
  version      
  software     
  developer    
  contact      
  link         
  doi          

  parameters: empty 


Metadata
  name: None
  author: None
  path: None
  date: 2025-07-02 14:18:32.415281
  description: test
  version: 1.0

Workflow
  No processing methods in the workflow.

Engine 
  name: None 
  author: None 
  path: None 
  date: 2025-07-02 14:18:32.416394 


Workflow
  No processing methods in the workflow.

Analyses 
  data_type:  
  formats: [] 
  data: 0 
  No data available. 



# Analyes

In [2]:
import numpy as np
from src.StreamPort.core import Engine
from src.StreamPort.core import Analyses

print("1: Creates an Analyses object and prints it")

ana1 = Analyses(data_type="unknown", formats=[], data = [
  {"name": "Analysis 1", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 2", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 3", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 4", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}
])

x = Engine(analyses=ana1)

x.print()


1: Creates an Analyses object and prints it

Engine 
  name: None 
  author: None 
  path: None 
  date: 2025-07-02 14:18:32.440194 


Workflow
  No processing methods in the workflow.

Analyses 
  data_type: unknown 
  formats: [] 
  data: 4 
    1. 3 items
    2. 3 items
    3. 3 items
    4. 3 items
 



# ProcessingMethod

In [3]:
from src.StreamPort.core import Engine
from src.StreamPort.core import ProcessingMethod

x = Engine()

print("1: Creates a ProcessingMethod object and prints it")
mt = ProcessingMethod()
print(mt)

print("2: Adds the settings to the Engine")
x.workflow.append(mt)
x.workflow.append(mt)
print("Number of methods: ", len(x.workflow))
print("\n")

print("3: Removes the settings from the Engine")
x.workflow.pop(1)
print("Number of methods: ", len(x.workflow))
print("\n")


1: Creates a ProcessingMethod object and prints it

ProcessingMethod
  data_type    
  method       
  required     
  algorithm    
  input        
  output       
  version      
  software     
  developer    
  contact      
  link         
  doi          

  parameters: empty 

2: Adds the settings to the Engine
Number of methods:  2


3: Removes the settings from the Engine
Number of methods:  1




# ProcessingMethod dispatchment

In [4]:
from src.StreamPort.core import Engine
from src.StreamPort.core import ProcessingMethod
from src.StreamPort.core import Analyses
import numpy as np


class UnkownNormalizeMinMax(ProcessingMethod):
    def __init__(self):
        super().__init__()
        self.data_type = "unknown"
        self.method = "Normalize"
        self.algorithm = "MinMax"
        self.input_instance = dict
        self.output_instance = dict
        self.number_permitted = float("inf")

    def run(self, analyses):
        res = analyses.data
        if len(res) > 0:
            for i in range(len(res)):
                ana = res[i]
                if not isinstance(ana, dict) or "y" not in ana:
                    print(
                        f"Skipping {ana['name']} because its data is not a dictionary with a 'y' key."
                    )
                    continue
                ana["y"] = (ana["y"] - ana["y"].min()) / (ana["y"].max() - ana["y"].min())
                res[i] = ana
            analyses.data = res
        else:
            print("No data to process.")
        return analyses


# Algorithm specific class
class UnkownNormalizeSNV(ProcessingMethod):
    def __init__(self, liftToZero=True):
        super().__init__()
        self.data_type = "unknown"
        self.method = "Normalize"
        self.algorithm = "standard_variance_normalization"
        self.input_instance = dict
        self.output_instance = dict
        self.number_permitted = float("inf")
        self.parameters = {"liftToZero": liftToZero}

    def run(self, analyses):
        res = analyses.data
        if len(res) > 0:
            for i in range(len(res)):
                ana = res[i]
                if not isinstance(ana, dict) or "y" not in ana:
                    print(
                        f"Skipping {ana['name']} because its data is not a dictionary with a 'y' key."
                    )
                    continue
                y = np.array(ana["y"])
                norm_data = (y - y.mean()) / y.std()
                if self.parameters["liftToZero"]:
                    norm_data += abs(norm_data.min())
                ana["y"] = norm_data
                res[i] = ana
            analyses.data = res
        else:
            print("No data to process.")
        return analyses

anas = Analyses(data_type="unknown", formats=[], data = [
  {"name": "Analysis 1", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 2", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 3", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 4", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}
])

x = Engine(analyses=anas)

print("1: Prints the analyses results in the Engine")
for res in x.analyses.data:
    print(f"Analyses: {res["name"]}")
    print(f"{"x"}: {res["x"]}")
    print(f"{"y"}: {res["y"]}")
    print("\n")

# Create an instance of the UnknownNormalizeMinMax class
norm_method = UnkownNormalizeMinMax()

# Create an instance of the UnknownNormalizeSNV class
norm_method2 = UnkownNormalizeSNV()

# Run the settings MinMax using the analyses as argument
analyses_minmax = norm_method.run(x.analyses)

print("2: Results from settings MinMax")
for key in analyses_minmax.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

# Run the settings SNV using the engine as argument
analyses_svn = norm_method2.run(x.analyses)

print("3: Results from settings snv")
for key in analyses_svn.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

# Adds method to the Engine
x.workflow.append(norm_method)

x.print()

x.run()

print("4: Results from the engine")
for key in x.analyses.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

1: Prints the analyses results in the Engine
Analyses: Analysis 1
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [65 69 83  2 92 86 47 56 98 87]


Analyses: Analysis 2
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [77 70 13 26 66 49 37  6 72 86]


Analyses: Analysis 3
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [13 63 30 75 73 65 69 75 59 42]


Analyses: Analysis 4
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [60 93 81 19 90 26 48 43 32 78]


2: Results from settings MinMax
Analyses: Analysis 1
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [0.65625    0.69791667 0.84375    0.         0.9375     0.875
 0.46875    0.5625     1.         0.88541667]


Analyses: Analysis 2
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [0.8875 0.8    0.0875 0.25   0.75   0.5375 0.3875 0.     0.825  1.    ]


Analyses: Analysis 3
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [0.         0.80645161 0.27419355 1.         0.96774194 0.83870968
 0.90322581 1.         0.74193548 0.46774194]


Analyses: Analysis 4
x: [ 1  2  3  4  5  6  7  8  9 10]
y: [0.55405405 1.  

# Class Inheritance

In [5]:
class Person:
  """Class docstrings go here."""

  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    """Class method docstrings go here."""
    print(self.firstname, self.lastname)

  def welcome_from_parent(self):
    print("Welcome from parent class")

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")

x.printname()

class Student(Person):
  def __init__(self, fname, lname, year):
    super().__init__(fname, lname)
    self.graduationyear = year

  def welcome(self):
    print("Welcome", self.firstname, self.lastname, "to the class of", self.graduationyear)

  def printname(self):
    print(self.firstname, self.lastname, "overwritten!")

  def printname_from_parent(self):
    super().printname()

x = Student("Mike", "Olsen", 2019)

x.printname()

x.printname_from_parent()

x.welcome()

x.welcome_from_parent()

#help(Person)

John Doe
Mike Olsen overwritten!
Mike Olsen
Welcome Mike Olsen to the class of 2019
Welcome from parent class
